<a href="https://colab.research.google.com/github/PavelStelmakhV/hw309-selection-of-hyperparameters/blob/main/hw309.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [106]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from keras import layers
from keras import callbacks
from keras import initializers
from keras import regularizers

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import BatchNormalization

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from tensorboard.plugins.hparams import api as hp

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical


# import numpy as np
# import matplotlib.pyplot as plt
# import tf_keras

# from keras import layers
# from keras import models
# from keras import regularizers
# from keras import callbacks
# from keras import initializers


# from keras.models import load_model


In [107]:
# !rm -rf ./logs/

In [108]:
num_classes = 10
log_dir = 'logs/hparam_tuning/'

In [109]:
%%capture
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [110]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

In [111]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([64, 128, 256]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.25, 0.5))
# HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.25]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['sgd', 'rmsprop', 'adam', 'nadam']))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([32, 64, 128, 256]))
HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([1, 2, 3, 4]))
HP_L2 = hp.HParam('l_2', hp.Discrete([1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER, HP_BATCH_SIZE, HP_NUM_LAYERS, HP_L2],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [112]:
def layer_relu(model, neurons, drop_out):
  w_init_relu = initializers.HeNormal(seed=42)
  b_init = initializers.Zeros()

  model.add(BatchNormalization())
  model.add(layers.Dense(neurons,
                        activation='relu',
                        kernel_initializer=w_init_relu,
                        bias_initializer=b_init,
                        kernel_regularizer=regularizers.L2(l2=hparams[HP_L2]),
                        # # kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2),
                        bias_regularizer=regularizers.L2(l2=hparams[HP_L2]),
                        # # bias_regularizer=regularizers.L1L2(l1=l1, l2=l2),
                        activity_regularizer=regularizers.L2(l2=hparams[HP_L2])
                        # # activity_regularizer=regularizers.L1L2(l1=l1, l2=l2)
                         )
  )
  model.add(layers.Dropout(drop_out))
  return model

In [113]:
def train_test_model(run_dir, hparams):

    model = Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    for num_layer in range(hparams[HP_NUM_LAYERS]):
      model = layer_relu(model, hparams[HP_NUM_UNITS], hparams[HP_DROPOUT])
      # print(num_layer, hparams[HP_NUM_LAYERS])
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='categorical_crossentropy',
        metrics=['accuracy'],
    )

    callback_list = [
        EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=6),
        ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True),
        TensorBoard(log_dir=run_dir, histogram_freq=1)
    ]
    # Train the model
    model.fit(x=x_train,
              y=y_train,
              epochs=2,
              batch_size=hparams[HP_BATCH_SIZE],
              validation_split=0.13,
              callbacks=callback_list,
              verbose=0
              )

    model.summary()

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print(f"Test Accuracy: {test_accuracy}")
    return test_accuracy


In [114]:
def run(run_dir, hparams, step):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(run_dir, hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=step)

In [ ]:
session_num = 10

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
  # for dropout_rate in HP_DROPOUT.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
      for batch_size in HP_BATCH_SIZE.domain.values:
        for num_layers in HP_NUM_LAYERS.domain.values:
          for l_2 in HP_L2.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
                HP_BATCH_SIZE: batch_size,
                HP_NUM_LAYERS: num_layers,
                HP_L2: l_2
            }
          run_name = "run-%d" % session_num
          run_name = run_name + f"[{optimizer}] d[{dropout_rate}] n[{num_units}] bs[{batch_size}] nl[{num_layers}] [{l_2}]"
          print('--- Starting trial: %s' % run_name)
          print({h.name: hparams[h] for h in hparams})

          run(log_dir + run_name, hparams, session_num)
          session_num += 1

--- Starting trial: run-10[adam] d[0.25] n[64] bs[32] nl[1] [0.01]
{'num_units': 64, 'dropout': 0.25, 'optimizer': 'adam', 'batch_size': 32, 'num_layers': 1, 'l_2': 0.01}


In [ ]:
# test_loss, test_accuracy = model.evaluate(x_test, y_test)
# print(f"Test Accuracy: {test_accuracy}")

In [ ]:
# model.summary()

In [ ]:
%tensorboard --logdir logs/hparam_tuning